<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_1023VDRJ1_1top60894_read_and_manage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語を読むための”ＴＭ語彙リスト”（総合版）Ver.4.0 の読み込み

* [松下言語学習ラボ](http://www17408ui.sakura.ne.jp/tatsum/database.html) 日本語を読むための語彙データベース Ver. 1.11 より
[重要度順語彙データベース (Top 60894) 重要度順位 00001-60894 (42MB)](http://www17408ui.sakura.ne.jp/tatsum/database/VDRJ_Ver1_1_Research_Top60894.xlsx) を利用する

In [ ]:
# Mac の retina ディスプレイの場合，高解像度の画面を使用する
%config InlineBackend.figure_format = 'retina'
try:
    import bit
except ImportError:
    !pip install ipynbname
    !git clone https://github.com/ShinAsakawa/bit.git
import bit
isColab = bit.isColab
HOME = bit.HOME
#if isColab:
#    !pip install transformers fugashi ipadic

In [ ]:
import os
import sys
import numpy as np
import random

# ローカルと colab との相違を吸収するために
# 本ファイルを Google Colaboratory 上で実行する場合に，必要となるライブラリをインストール
if isColab:
    #!git clone https://github.com/ShinAsakawa/ccap.git

    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3 > /dev/null 2>&1
    !pip install --upgrade ipadic
    !python -m unidic download

    !pip install --upgrade openpyxl    # エクセルファイルを読むこむ際にバージョンの相違で動作しない場合があるので念の為
    !pip install --upgrade pandas      # 同上
    
    !pip install japanize_matplotlib
    !pip install jaconv

## データの作成

In [ ]:
%%time
import numpy as np
import requests
import pandas as pd
import os
import jaconv
import datetime

def make_VDRJ_dict(cover_rate:float=0.99,
                   verbose:bool=False,
                  ):
    """
    日本語を読むための語彙データベース（VDRJ） Ver. 1.1　（＝日本語を読むための”ＴＭ語彙リスト”（総合版）　Ver.4.0）
    を読み込んで，「単語」と対応する「読み」のリストを返す
    
    引数:
        cover_rate:float
            松下コーパスの `(Fw)累積テキストカバー率（想定既知語彙分を含む)` に基づいて語彙を選択するためのカバー率
            デフォルトでは 0.99 だが変更可能
    戻り値:
        X:dict 以下のキーを持つ辞書
             'lexeme': 語彙素
              'orth': 書記素
              'yomi': よみ
              'mora': よみのモーラ
              'cover_r': 累積カバー率
            
    """
    
    vdrj_url='http://www17408ui.sakura.ne.jp/tatsum/database/VDRJ_Ver1_1_Research_Top60894.xlsx'
    excel_fname = vdrj_url.split('/')[-1]  # 直上行の url からエクセルファイル名を切り出す

    if not os.path.exists(excel_fname): # もしエクセルファイルが存在しなかったら ダウンロード
        print(f'エクセルファイルのダウンロード {datetime.datetime.now()}...') if verbose else None
        r = requests.get(vdrj_url)
        with open(excel_fname, 'wb') as f:
            total_length = int(r.headers.get('content-length'))
            print('Downloading {0} - {1} bytes'.format(excel_fname, (total_length)))
            f.write(r.content)
        print(f'done {datetime.datetime.now()}') if verbose else None
    
    # 実際のエクセルファイルの読み込み
    sheet_name='重要度順語彙リスト60894語'  # シート名を指定
    print(f'エクセルファイルの読み込み {datetime.datetime.now()}...') if verbose else None
    df = pd.read_excel(excel_fname, sheet_name=sheet_name)
    print(f'done. {datetime.datetime.now()}') if verbose else None

    print(f'データ作成 {datetime.datetime.now()}...') if verbose else None
    # 累積カバー率が `cover_rate` 以下の語を選択
    df = df[df['(Fw)累積テキストカバー率（想定既知語彙分を含む）\nFw Cumulative Text Coverage including Assumed Known Words']<cover_rate]
    
    # 品詞 POS が '名詞-普通名詞-一般' のみを抽出
    df = df[df['品詞\nPart of Speech']=='名詞-普通名詞-一般']
    
    # NaN を含む行を削除
    #df = df[df['標準的読み方（カタカナ）\nStandard Reading (Katakana)'] != pd.np.nan]
    #df = df[df['標準的（新聞）表記\nStandard (Newspaper) Orthography'] != pd.np.nan]
    df = df[['見出し語彙素\nLexeme',
             '標準的（新聞）表記\nStandard (Newspaper) Orthography',
             '標準的読み方（カタカナ）\nStandard Reading (Katakana)', 
             '品詞\nPart of Speech',
             '(Fw)累積テキストカバー率（想定既知語彙分を含む）\nFw Cumulative Text Coverage including Assumed Known Words',
             'ID',
            ]].dropna()
    
    # 必要となる情報のみをリスト化: ['語彙素', '書記素', 'よみ', '累積カバー率', 'ID']
    Lexeme = df['見出し語彙素\nLexeme'].to_list()
    CoverR = df['(Fw)累積テキストカバー率（想定既知語彙分を含む）\nFw Cumulative Text Coverage including Assumed Known Words'].to_list()
    ID     = df['ID'].to_list()
    Ortho  = df['標準的（新聞）表記\nStandard (Newspaper) Orthography'].to_list()
    Yomi   = df['標準的読み方（カタカナ）\nStandard Reading (Katakana)'].to_list()

    # 結果を Python の辞書にまとめる
    X, orth_cnt = {}, {}
    fish3words = []
    for l, o, y, r, idx in zip(Lexeme, Ortho, Yomi, CoverR, ID):
        
        # `よみ` に '/' が含まれる項目 `ALT` を `オルト/エイエルティー` の場合最初のエントリだけを採用する
        if isinstance(y, str):
            y = jaconv.normalize(y)
            if '/' in y:
                y = y.split('/')[0]
                print(f'{y}, idx:{idx}')
                
        # `語彙素` に '鱻' が含まれる項目は '鱻' 以降を切り捨てる
        if isinstance(l, str):
            if '鱻' in l:
                fish3words.append({'idx':idx, 'lexeme':l})
                l = l.split('鱻')[0]
                
        if isinstance(l, str):
            lexeme = jaconv.normalize(l)  # 書記素を UTF-8 NKCD に正規化
        else:
            continue

        if isinstance(o, str):
            orth = jaconv.normalize(o)
            
            # 書記素が '*' になっている場合があるので，語彙素情報で置き換える
            orth = lexeme if orth == '*' else orth
        else:
            continue

        if isinstance(y, str):
            yomi   = jaconv.hira2kata(y)
            hira = jaconv.kata2hira(yomi)
            mora = jaconv.hiragana2julius(hira).split(' ')
        else:
            continue
        
        # lexeme = jaconv.normalize(l)  # 書記素を UTF-8 NKCD に正規化
        # orth   = jaconv.normalize(o)  # 形態素を UTF-8 NKCD に正規化
        # yomi   = jaconv.hira2kata(y)  #           
        # hira = jaconv.kata2hira(y)
        # mora = jaconv.hiragana2julius(hira).split(' ')
        
        X[idx] = {'lexeme': lexeme,
                  'orth': orth,
                  'yomi': yomi,
                  'mora': mora,
                  'cover_r': r,
                 }
        # if X[idx]['orth'] == '*':
        #         # 書記素情報が '*' になっている場合があるので，語彙素情報で置き換える
        #         X[idx]['orth'] = X[idx]['lexeme']

    print(fish3words)
    print(f'done {datetime.datetime.now()}...') if verbose else None
    return X, orth_cnt

# 下行で，例えば cover_rate=0.95 とすれば，累積カバー率 95 % のデータを得る
X, orth_cnt = make_VDRJ_dict(cover_rate=0.99, verbose=True)

## 読み込んだ松下語彙データの確認. 基本統計量の計算

In [ ]:
import operator
import matplotlib.pyplot as plt
import japanize_matplotlib

# モーラ，よみ，語彙素，書記素，のそれぞれについて頻度グラフを描画
for key in ['mora', 'yomi', 'lexeme', 'orth']:
    cont = {}
    for k,v in X.items():
        for ch in X[k][key]:
            if not ch in cont:
                cont[ch] = 1
            else:
                cont[ch] += 1

    cont_sorted = sorted(cont.items(), key=operator.itemgetter(1),reverse=True)
    
    plt.figure(figsize=(14,4))
    plt.bar(range(len(cont_sorted)), [x[1] for x in cont_sorted])
    plt.xticks(ticks=range(len(cont_sorted)), labels=[c[0] for c in cont_sorted])
    plt.title(f'{key} 頻度')
    plt.show()
    #print(cont_sorted)

for key in ['mora', 'yomi', 'lexeme', 'orth']:
    cont = {}
    for k,v in X.items():
        _len = len(X[k][key])
        if not _len in cont:
            cont[_len] = 1
        else:
            cont[_len] += 1
    cont_sorted = sorted(cont.items(), key=operator.itemgetter(0))
    #cont_sorted = sorted(cont.items(), key=operator.itemgetter(1),reverse=True)
    
    plt.figure(figsize=(14,4))
    plt.bar(range(len(cont_sorted)), [x[1] for x in cont_sorted])
    plt.xticks(ticks=range(len(cont_sorted)), labels=[c[0] for c in cont_sorted])
    plt.title(f'語長 {key} 頻度')
    plt.show()
    print(cont_sorted)
    
    max_length = cont_sorted[-1][0]
    print(f'max_length:{max_length}')
    for k,v in X.items():
        _len = len(X[k][key])
        if _len == max_length:
            print(f'{k} 最長項目:{X[k]}') 